# EDA

## Load

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from src.config import DATA_RAW_DIR, DATA_RAW_FILENAME

In [ ]:
df_raw = pd.read_csv(os.path.join(DATA_RAW_DIR, DATA_RAW_FILENAME), sep=";")

## First glance at raw data

In [ ]:
display(df_raw.shape)

In [ ]:
display(df_raw.info())

In [ ]:
df_raw.isna().sum()

In [ ]:
df_raw[
    df_raw["sol_prod"].isna() & df_raw["sol_prod_1"].notna() & df_raw["sol_prod_1"] > 0
][["timestamp", "sol_prod", "sol_prod_1", "sol_prod_2", "sol_prod_3"]].head(10)

There are missing values in the `sol_prod` column, but the `sol_prod_1` column has values. The values in `sol_prod` should be recoverable from `sol_prod_1` and `sol_prod_2`.

In [ ]:
df_raw[
    df_raw["wb_tot_charge"].isna() & df_raw["wb_0_tot_charge"].notna() & df_raw["wb_0_tot_charge"] > 0
][["timestamp", "wb_tot_charge", "wb_0_tot_charge", "wb_0_grid_cons", "wb_0_sol_charge"]].head(10)

In [ ]:
df_raw[
    df_raw["wb_tot_charge"].notna() & df_raw["wb_0_tot_charge"].notna() & df_raw["wb_0_tot_charge"] > 0
][["timestamp", "wb_tot_charge", "wb_0_tot_charge", "wb_0_grid_cons", "wb_0_sol_charge"]].head(10)

There are also missing values in the `wb_tot_charge` column, but the `wb_0_tot_charge` column has values. The values in `wb_tot_charge` should be recoverable from `wb_0_tot_charge`.

## Info, Describe, Overview

In [ ]:
df_raw["timestamp"] = pd.to_datetime(df_raw["timestamp"], format="%Y-%m-%d %H:%M")

In [ ]:
df_raw.info()

In [ ]:
display(df_raw.loc[:, df_raw.select_dtypes(include="number").columns].describe().T)

In [ ]:
from src.eda import overview

overview(df_raw)

### Observation
* `sol_prod_3` has only zeros.
* `wb_0_grid_cons` has negative values.

## Correlations

In [ ]:
df_raw_copy = df_raw.loc[:, df_raw.select_dtypes(include=["number"]).columns].copy()


corr = df_raw_copy.corr()
mask = np.abs(corr) < 0.3
annot = corr.round(2).astype(str)
annot[mask] = ""

plt.figure(figsize=(9, 7))
sns.heatmap(
    corr,
    annot=annot,
    fmt="",
    cmap="coolwarm",
    vmin=-1,
    vmax=1,
)

plt.title("Correlations between numerical columns", y=1.02)
plt.show()

del df_raw_copy

## Outlier Detection

In [ ]:
from src.eda import mark_outliers_mad

# Mark outliers in the raw data using the Median Absolute Deviation (MAD) method
# deviations from the median are considered outliers if they exceed 3 times the median absolute deviation
# Note the log scale
mark_outliers_mad(
    df_raw,
    )

### Observations
* Most of the columns have values that are considered outliers by the mad-3 method. Most of the values are very small, so this is not surprising considering night times with low energy production and consumption are also included.

## Grouped Series

In [ ]:
df_time = df_raw.copy()
df_time = df_time.drop(columns=["sol_prod_3"])
df_time["year"] = df_time["timestamp"].dt.year
df_time["month"] = df_time["timestamp"].dt.month
df_time["day"] = df_time["timestamp"].dt.day
df_time["hour"] = df_time["timestamp"].dt.hour
df_time["minute"] = df_time["timestamp"].dt.minute
df_time["day_of_week"] = df_time["timestamp"].dt.dayofweek
df_time["day_of_year"] = df_time["timestamp"].dt.dayofyear
df_time["week_of_year"] = df_time["timestamp"].dt.isocalendar().week
df_time = df_time.set_index("timestamp").sort_index()

In [ ]:
df_time.loc[df_time["sol_prod"].isna(), "sol_prod"] = df_time[df_time["sol_prod"].isna()]["sol_prod_1"] + df_time[df_time["sol_prod"].isna()]["sol_prod_2"] 


### By year and day

In [ ]:
drop_cols = ["month", "day", "hour", "minute", "day_of_week", "week_of_year"]
group_cols = ["installation", "year", "day_of_year"]

agg_dict = {"soc": "mean"} | {
    c: "sum" for c in df_time.columns if c not in drop_cols + group_cols + ["soc"]
}

df_by_year_day = (
    df_time.drop(columns=drop_cols)
    .groupby(group_cols)
    .agg(agg_dict)
    .reset_index()
)

df_by_year_day["installation_year"] = df_by_year_day[["installation", "year"]].apply(
    lambda x: f"{x["installation"]} {x["year"]}", axis=1
)

# display(df_by_year_day.head())

show_columns = [
    "soc",
    "bat_charge",
    "bat_discharge",
    "grid_feed_in",
    "grid_cons",
    "sol_prod_1",
    "sol_prod_2",
    "sol_prod",
    "house_cons",
    "add_cons",
    "ext_cons",
    "tot_cons",
    "wb_0_tot_charge",
    "wb_0_grid_cons",
    "wb_0_sol_charge",
]


fig, axs = plt.subplots(
    nrows=len(show_columns), figsize=(16, 2 * len(show_columns) + 2)
)

for col, ax in zip(show_columns, axs):
    sns.lineplot(
        data=df_by_year_day,
        x="day_of_year",
        y=col,
        hue=df_by_year_day["installation_year"],
        linewidth=0.5,
        # palette="tab10",
        ax=ax,
    )
    ax.legend(title="Installation Year", bbox_to_anchor=(1.01, 1), loc="upper left")
    ax.set_title(col)
    ax.set_xticks(np.arange(0, 366, 30))
    ax.set_xticklabels(
        [f"{i:01d}" for i in np.arange(0, 366, 30)],
    )
    if col == "soc":
        ticks = np.arange(0, 101, 20)
        ax.set_ylim(0, 100)
        ax.set_yticks(ticks)
        ax.set_yticklabels([f"{i}%" for i in ticks])
        ax.set_ylabel("")
    else:
        ax.set_ylabel("Wh")

    ax.yaxis.grid(True, which="major", linestyle="--", color="lightgrey")


plt.suptitle("Daily Total Power Consumption and Production by Installation and Year", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

#### Observations
* The yearly cycle of the power consumption and production is visible.
* The wallbox has been installed in early 2024.
* In 2022 there is a high external and additional consumption spike.

### By year and weekday

In [ ]:
drop_cols = ["month", "day", "hour", "minute", "day_of_year"]
group_cols = ["installation", "year", "week_of_year", "day_of_week"]

agg_dict = {"soc": "mean"} | {
    c: "sum" for c in df_time.columns if c not in drop_cols + group_cols + ["soc"]
}

df_by_year_weekday = (
    df_time.drop(columns=drop_cols)
    .groupby(group_cols)
    .agg(agg_dict)
    .reset_index()
)

df_by_year_weekday["installation_year"] = df_by_year_weekday[["installation", "year"]].apply(
    lambda x: f"{x["installation"]} {x["year"]}", axis=1
)

# display(df_by_year_weekday.head())

show_columns = [
    "soc",
    "bat_charge",
    "bat_discharge",
    "grid_feed_in",
    "grid_cons",
    "sol_prod_1",
    "sol_prod_2",
    "sol_prod",
    "house_cons",
    "add_cons",
    "ext_cons",
    "tot_cons",
    "wb_0_tot_charge",
    "wb_0_grid_cons",
    "wb_0_sol_charge",
]


fig, axall = plt.subplots(
    nrows=len(show_columns),
    ncols=7,
    figsize=(16, 2 * len(show_columns) + 2),
    sharey='row'
)

for col, axrow in zip(show_columns, axall):
    weekdays = sorted(df_by_year_weekday["day_of_week"].unique())
    for day_of_week, ax in zip(weekdays, axrow):
        sns.lineplot(
            data=df_by_year_weekday[df_by_year_weekday["day_of_week"] == day_of_week],
            x="week_of_year",
            y=col,
            hue=df_by_year_weekday["installation_year"],
            linewidth=0.5,
            # palette="tab10",
            ax=ax,
        )
        ax.legend(title="Installation Year", bbox_to_anchor=(1.01, 1), loc="upper left")
        ax.set_title(col)
        ax.set_xticks(np.arange(0, 53, 10))
        ax.set_xticklabels(
            [f"{i:01d}" for i in np.arange(0, 53, 10)],
        )
        ax.set_xlabel("")

        if col == "soc":
            ticks = np.arange(0, 101, 20)
            ax.set_ylim(0, 100)
            ax.set_yticks(ticks)
            ax.set_yticklabels([f"{i}%" for i in ticks])
            ax.set_ylabel("")
        else:
            ax.set_ylabel("Wh")

        ax.yaxis.grid(True, which="major", linestyle="--", color="lightgrey")

        # Only show legend for last day of week
        if day_of_week == weekdays[-1]:
            ax.legend(title="Installation Year", bbox_to_anchor=(1.01, 1), loc="upper left")
        else:
            ax.legend().remove()
            



plt.suptitle(
    "Weekday Power Consumption and Production by Installation and Year",
    fontsize=16,
    y=1.02,
)
plt.tight_layout()
plt.show()

### Rolling Mean (4-Week) Of Daily Power Consumption and Production by Weekday

In [ ]:
drop_cols = ["month", "day", "hour", "minute", "day_of_year", "year"]
group_cols = ["installation", "day_of_week", "week_of_year"]

agg_dict = {"soc": "mean"} | {
    c: "sum" for c in df_time.columns if c not in drop_cols + group_cols + ["soc"]
}


df_agg = (
    df_time.drop(columns=drop_cols).groupby(group_cols, as_index=False).agg(agg_dict)
)


value_cols = [c for c in df_agg.columns if c not in group_cols]
df_agg[value_cols] = (
    df_agg.groupby("installation", group_keys=False)[value_cols]
          .rolling(window=4, min_periods=1)
          .mean()
          .reset_index(level=0, drop=True)
)

df_by_year_weekday = df_agg.reset_index(drop=True)

df_by_year_weekday["installation_dow"] = df_by_year_weekday[
    ["installation", "day_of_week"]
].apply(lambda x: f"day {x["day_of_week"]}", axis=1)

display(df_by_year_weekday.head())

show_columns = [
    "soc",
    "bat_charge",
    "bat_discharge",
    "grid_feed_in",
    "grid_cons",
    "sol_prod_1",
    "sol_prod_2",
    "sol_prod",
    "house_cons",
    "add_cons",
    "ext_cons",
    "tot_cons",
    "wb_0_tot_charge",
    "wb_0_grid_cons",
    "wb_0_sol_charge",
]


fig, axall = plt.subplots(
    nrows=len(show_columns),
    figsize=(16, 2 * len(show_columns) + 2),
)

for col, axrow in zip(show_columns, axall):
    ax = axrow
    sns.lineplot(
        data=df_by_year_weekday,
        x="week_of_year",
        y=col,
        hue=df_by_year_weekday["installation_dow"],
        linewidth=1,
        ax=ax,
    )
    ax.set_title(f"{col}")
    if col == "soc":
        ticks = np.arange(0, 101, 20)
        ax.set_ylim(0, 100)
        ax.set_yticks(ticks)
        ax.set_yticklabels([f"{i}%" for i in ticks])
        for tick in ticks:
            ax.axhline(y=tick, linestyle="--", linewidth=0.5, color="gray")
    else:
        ax.set_ylabel("Wh")

    if day_of_week == weekdays[-1]:
        ax.legend(title="Day of Week", bbox_to_anchor=(1.01, 0.5), loc="center left")
    else:
        ax.legend().remove()

plt.suptitle(
    "Rolling Mean (4-Week) Of Daily Power Consumption and Production by Weekday",
    fontsize=16,
    y=1.02,
)
plt.tight_layout()
plt.show()

#### Observations
* A significant difference in power consumption is not observed between weekdays and weekends.
* The consumption spike around weeks 28-30 are still visible in the rolling mean.
* The battery was never fully charged, indicating a potential need for increased solar production or reduced consumption.

### Rolling Mean (12-Week) Of Daily Power Consumption and Production by Weekday

In [ ]:
drop_cols = ["month", "day", "hour", "minute", "day_of_year", "year"]
group_cols = ["installation", "day_of_week", "week_of_year"]

agg_dict = {"soc": "mean"} | {
    c: "sum" for c in df_time.columns if c not in drop_cols + group_cols + ["soc"]
}


df_agg = (
    df_time.drop(columns=drop_cols).groupby(group_cols, as_index=False).agg(agg_dict)
)


value_cols = [c for c in df_agg.columns if c not in group_cols]
df_agg[value_cols] = (
    df_agg.groupby("installation", group_keys=False)[value_cols]
          .rolling(window=12, min_periods=1)
          .mean()
          .reset_index(level=0, drop=True)
)

df_by_year_weekday = df_agg.reset_index(drop=True)

df_by_year_weekday["installation_dow"] = df_by_year_weekday[
    ["installation", "day_of_week"]
].apply(lambda x: f"day {x["day_of_week"]}", axis=1)

display(df_by_year_weekday.head())

show_columns = [
    "soc",
    "bat_charge",
    "bat_discharge",
    "grid_feed_in",
    "grid_cons",
    "sol_prod_1",
    "sol_prod_2",
    "sol_prod",
    "house_cons",
    "add_cons",
    "ext_cons",
    "tot_cons",
    "wb_0_tot_charge",
    "wb_0_grid_cons",
    "wb_0_sol_charge",
    
]


fig, axall = plt.subplots(
    nrows=len(show_columns),
    figsize=(16, 2 * len(show_columns) + 2),
)

for col, axrow in zip(show_columns, axall):
    ax = axrow
    sns.lineplot(
        data=df_by_year_weekday,
        x="week_of_year",
        y=col,
        hue=df_by_year_weekday["installation_dow"],
        linewidth=1,
        ax=ax,
    )
    ax.set_title(f"{col}")
    if col == "soc":
        ticks = np.arange(0, 101, 20)
        ax.set_ylim(0, 100)
        ax.set_yticks(ticks)
        ax.set_yticklabels([f"{i}%" for i in ticks])
        for tick in ticks:
            ax.axhline(y=tick, linestyle="--", linewidth=0.5, color="gray")
    else:
        ax.set_ylabel("Wh")

    if day_of_week == weekdays[-1]:
        ax.legend(title="Day of Week", bbox_to_anchor=(1.01, 0.5), loc="center left")
    else:
        ax.legend().remove()

plt.suptitle(
    "Rolling Mean (12-Week) Of Daily Power Consumption and Production by Weekday",
    fontsize=16,
    y=1.02,
)
plt.tight_layout()
plt.show()

In [ ]:

df_by_year_weekday = (
    df_time.reset_index().drop(columns=["month", "day", "hour", "minute", "day_of_year", "year", "timestamp"])
    .groupby(["installation", "week_of_year", "day_of_week"]).sum()
    .rolling(window=4, min_periods=1)
    .mean()
    .reset_index()
)

display(df_time.reset_index().drop(columns=["month", "day", "hour", "minute", "day_of_year", "year", "timestamp"])
    .groupby(["installation", "week_of_year", "day_of_week"]).sum().head(5))

display(    df_time.reset_index().drop(columns=["month", "day", "hour", "minute", "day_of_year", "year", "timestamp"])
    .groupby(["installation", "week_of_year", "day_of_week"]).sum()
    .rolling(window=4, min_periods=1)
    .mean().head(5))




# display(df_time.reset_index()[(df_time.reset_index()["week_of_year"]==1) & (df_time.reset_index()["day_of_week"]==0)].head(5))
# display(df_time.reset_index().drop(columns=["month", "day", "hour", "minute", "day_of_year", "year", "timestamp"]).head(5))
# display(df_time.reset_index().drop(columns=["month", "day", "hour", "minute", "day_of_year", "year", "timestamp"]).groupby(["installation", "week_of_year", "day_of_week"]).head(5))
# display(df_time.reset_index().drop(columns=["month", "day", "hour", "minute", "day_of_year", "year", "timestamp"]).groupby(["installation", "week_of_year", "day_of_week"]).rolling(window=4, min_periods=1).mean().head(5))
# display(df_time.reset_index().drop(columns=["month", "day", "hour", "minute", "day_of_year", "year", "timestamp"]).groupby(["installation", "week_of_year", "day_of_week"]).rolling(window=4, min_periods=1).mean().reset_index().head(5))
